<a href="https://colab.research.google.com/github/nosgueira/PLN-2022-1/blob/main/SentimentAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Polaridade Textos

In [9]:
import pandas as pd
import math

In [3]:
df = pd.read_csv('https://raw.githubusercontent.com/viniciusrpb/cic0269_natural_language_processing/main/datasets/corpora/training.csv', sep='\t')
df

,Id,text,Expeccted
0,0,"Ótima localização, bom atendimento, bom sinal ...",neutro
1,1,"O hotel é bem localizado e silencioso, o que c...",neutro
2,2,O hotel é muito bom. O atendimento é excelente...,positivo
3,3,"Hotel antigo, mal conservado e mal reformado, ...",negativo
4,4,"Hotel bem localizado, perto de dois shopping, ...",positivo
...,...,...,...
4995,4995,"Hotel muito bem localizado, com shopping e dem...",positivo
4996,4996,Além de oferecer uma boa hospedagem com confor...,positivo
4997,4997,"Hotel charmoso, bom atendimento, quartos confo...",positivo
4998,4998,"O hotel fica super bem localizado, em frente a...",positivo


In [6]:
textos = df.text.to_list()

## Representação TFIDF

In [8]:
def preproc(text):
    return text.split()

### Calculando Bag of Words

In [15]:
bow_dict = {}
for i,doc in enumerate(textos):
    doc_name = df.Id[i]
    bow_dict[doc_name] = dict()
    for word in preproc(doc):
        bow_dict[doc_name][word] = 1 if word not in bow_dict[doc_name] else bow_dict[doc_name][word]+1
        
bow = pd.DataFrame().from_records(bow_dict).fillna(0).T.astype(int)

In [18]:
tf = {}

for i, doc in enumerate(textos):

    doc_name = df.Id[i]
    tf[doc_name] = dict()
    words = preproc(doc)

    for word in words:
        tf[doc_name][word] = (1 if  word not in tf[doc_name] 
                                    else tf[doc_name][word]+1)
    tf[doc_name] = {word:(freq/len(words)) 
                    for word, freq in tf[doc_name].items()}

idf =  {}

vocab = bow.columns
idf =  {}

for word in vocab:
    idf[word] = math.log(len(bow)/len(bow[word][bow[word]!=0]))

tfidf_dict = {}

for doc in tf:
    tfidf_dict[doc] = dict()
    for word in tf[doc]:
        tfidf_dict[doc][word] = tf[doc][word]*idf[word]

tfidf = pd.DataFrame().from_records(tfidf_dict).fillna(0).T

In [19]:
tfidf

,Ótima,"localização,",bom,"atendimento,",sinal,de,wi-fi,"(gratuito),",preço.,O,...,"(fonte,",artesanato),docinho,Tranquila.,formas,"Sara,","Cidade),",selecionada.,Profissionalismo.,quentinhos.
0,0.035984,0.024501,0.076604,0.031674,0.046421,0.011370,0.041321,0.084329,0.052863,0.014791,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.008202,0.000000,0.000000,0.000000,0.013339,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.057459,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.014354,0.000000,0.000000,0.000000,0.015562,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,0.000000,0.000000,0.051580,0.106636,0.000000,0.015311,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4996,0.000000,0.000000,0.000000,0.000000,0.000000,0.012759,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4997,0.000000,0.000000,0.048356,0.099971,0.000000,0.007177,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.266162
4998,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.044818,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
